In [3]:
import cv2
import mediapipe as mp
import pyautogui
import time
import numpy as np
import threading
import queue

class FrameGrabber(threading.Thread):
    def __init__(self, video_source, frame_queue):
        super().__init__()
        self.video = cv2.VideoCapture(video_source)
        self.frame_queue = frame_queue
        self.daemon = True
        self.running = True

    def run(self):
        while self.running:
            ret, frame = self.video.read()
            if not ret:
                print("Failed to grab a frame.")
                self.running = False
            else:
                if self.frame_queue.qsize() < 2:  # Avoids overcrowding the queue
                    self.frame_queue.put(frame)

    def stop(self):
        self.running = False
        self.video.release()

frame_queue = queue.Queue()
video_source = 1
frame_grabber = FrameGrabber(video_source, frame_queue)
frame_grabber.start()

handGesture = mp.solutions.hands.Hands(max_num_hands=1, min_detection_confidence=0.5)
drawingTools = mp.solutions.drawing_utils
screenWidth, screenHeight = pyautogui.size()

prev_time = time.time()
process_every_n_frames = 2  # Adjust based on performance
frame_count = 0

try:
    while True:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame = cv2.flip(frame, 1)
            frame = cv2.resize(frame, (640, 480))  # Reduce frame resolution
            frameHeight, frameWidth, _ = frame.shape
            rgbConvertedFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            if frame_count % process_every_n_frames == 0:
                output = handGesture.process(rgbConvertedFrame)
                hands = output.multi_hand_landmarks

                if hands:
                    for hand in hands:
                        drawingTools.draw_landmarks(frame, hand, mp.solutions.hands.HAND_CONNECTIONS)

            frame_count += 1

            curr_time = time.time()
            fps = 1 / (curr_time - prev_time)
            prev_time = curr_time
            cv2.putText(frame, f'FPS: {int(fps)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            cv2.imshow('Virtual Mouse', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
finally:
    frame_grabber.stop()
    cv2.destroyAllWindows()


KeyboardInterrupt: 

In [4]:
import cv2
import mediapipe as mp
import pyautogui
import time
import numpy as np
import threading
import queue

class FrameGrabber(threading.Thread):
    def __init__(self, video_source, frame_queue):
        super().__init__()
        self.video = cv2.VideoCapture(video_source)
        self.frame_queue = frame_queue
        self.daemon = True
        self.running = True

    def run(self):
        while self.running:
            ret, frame = self.video.read()
            if not ret:
                print("Failed to grab a frame.")
                self.running = False
            else:
                if self.frame_queue.qsize() < 2:  # Avoid overcrowding the queue
                    self.frame_queue.put(frame)

    def stop(self):
        self.running = False
        self.video.release()

frame_queue = queue.Queue()
video_source = 1
frame_grabber = FrameGrabber(video_source, frame_queue)
frame_grabber.start()

handGesture = mp.solutions.hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
drawingTools = mp.solutions.drawing_utils
screenWidth, screenHeight = pyautogui.size()

prev_time = time.time()
process_every_n_frames = 2  # Adjust based on performance
frame_count = 0

try:
    while True:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame = cv2.flip(frame, 1)
            frame = cv2.resize(frame, (640, 480))  # Reduce frame resolution
            frameHeight, frameWidth, _ = frame.shape
            rgbConvertedFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            if frame_count % process_every_n_frames == 0:
                output = handGesture.process(rgbConvertedFrame)
                hands = output.multi_hand_landmarks

                if hands:
                    for hand in hands:
                        drawingTools.draw_landmarks(frame, hand, mp.solutions.hands.HAND_CONNECTIONS)
                        landmarks = hand.landmark
                        x8, y8 = int(landmarks[8].x * frameWidth), int(landmarks[8].y * frameHeight)
                        x4, y4 = int(landmarks[4].x * frameWidth), int(landmarks[4].y * frameHeight)
                        x, y = (x8 + x4) // 2, (y8 + y4) // 2
                        cv2.circle(frame, (x, y), 5, (0, 255, 255), -1)
                        mousePositionX = np.interp(x, (0, frameWidth), (0, screenWidth))
                        mousePositionY = np.interp(y, (0, frameHeight), (0, screenHeight))
                        pyautogui.moveTo(mousePositionX, mousePositionY)

            frame_count += 1

            curr_time = time.time()
            fps = 1 / (curr_time - prev_time)
            prev_time = curr_time
            cv2.putText(frame, f'FPS: {int(fps)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            cv2.imshow('Virtual Mouse', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
finally:
    frame_grabber.stop()
    cv2.destroyAllWindows()


Failed to grab a frame.
